# Ejecución de pruebas sobre QUIJOTE

En el siguiente código se generan los mapas necesarios y se ejecutan las pruebas sobre los mapas de QUIJOTE a diferentes frecuencias

In [11]:
import pandas as pd
import numpy as np
import astropy
import healpy as hp
import tensorflow as tf
import tools
import numpy as np
import pandas as pd
from scipy.stats import bootstrap
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
import tensorflow as tf

## Carga de mapas

Se cargan los mapas de QUIJOTE en todas las frecuencias y se les aplica el suavizado necesario

In [12]:
mapa11 = tools.leer_mapa('../datasets/quijote/quijote11.fits')
mapa13 = tools.leer_mapa('../datasets/quijote/quijote13.fits')
mapa17 = tools.leer_mapa('../datasets/quijote/quijote17.fits')
mapa19 = tools.leer_mapa('../datasets/quijote/quijote19.fits')
fwhm_degrees = 0.917 
fwhm_radians = np.deg2rad(fwhm_degrees)
# Suavizar el mapa
mapa11 = hp.smoothing(mapa11, fwhm=fwhm_radians)
fwhm_degrees = 0.917 
fwhm_radians = np.deg2rad(fwhm_degrees)
# Suavizar el mapa
mapa13 = hp.smoothing(mapa13, fwhm=fwhm_radians)
fwhm_degrees = 0.6491
fwhm_radians = np.deg2rad(fwhm_degrees)
# Suavizar el mapa
mapa17 = hp.smoothing(mapa17, fwhm=fwhm_radians)
fwhm_degrees = 0.672
fwhm_radians = np.deg2rad(fwhm_degrees)
# Suavizar el mapa
mapa19 = hp.smoothing(mapa19, fwhm=fwhm_radians)

/home/studio-lab-user/testcode/code/tools.py:23: HealpyDeprecationWarning: "verbose" was deprecated in version 1.15.0 and will be removed in a future version. 
  mapa_cmb, header = hp.read_map(ruta_archivo_fits, field=campo, h=True, verbose=False)


## Carga de catálogos

Se cargan los catálogos principales y extendidos de QUIJOTE, estos catálogos no se dividen en frecuencias

In [13]:
main = pd.read_csv('../datasets/quijote/catalogoQui2.csv',sep =';')
ext = pd.read_csv('../datasets/quijote/catalogoQui.csv',sep =';')

## Generación de imágenes

Se utiliza la función generadora de imágenes y se adapta a QUIJOTE. Se aplica sobre los cuatro mapas para ambos catálogos y se guardan en un directorio

In [14]:
def quijote_image_generator(mapa,catalogue,path):
    for glat,glon,name in zip(catalogue.glat,catalogue.glon,catalogue.name):
            img = tools.extraer_submapa_cmb(mapa,lat = glat,lon = glon,N = 64,res = 6.86,vmin = None, retornar_imagen=True)
             # Accede a la máscara
            tools.guardar_imagen_jpg(img,f'{path}/{name}.jpg')

In [ ]:
quijote_image_generator(mapa11,ext,'../images/quijote/images_quijote_11_ext')
quijote_image_generator(mapa13,ext,'../images/quijote/images_quijote_13_ext/')
quijote_image_generator(mapa17,ext,'../images/quijote/images_quijote_17_ext/')
quijote_image_generator(mapa19,ext,'../images/quijote/images_quijotek_19_ext/')

In [16]:
quijote_image_generator(mapa11,main,'../images/quijote/images_quijote_11/')
quijote_image_generator(mapa13,main,'../images/quijote/images_quijote_13/')
quijote_image_generator(mapa17,main,'../images/quijote/images_quijote_17/')
quijote_image_generator(mapa19,main,'../images/quijote/images_quijote_19/')

In [17]:
model = tf.keras.models.load_model('../models/MCD1504_6.h5')

activar la GPU en caso de que se pueda

In [18]:


# Verifica la disponibilidad de GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
print("GPUs disponibles: ", len(gpus))

# Configura TensorFlow para que use la GPU si está disponible
if gpus:
    try:
        # Establece la memoria de la GPU para que crezca según sea necesario.
        # Esto puede prevenir errores de asignación de memoria en TensorFlow.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

GPUs disponibles:  1
Physical devices cannot be modified after being initialized


## Ejecución de pruebas

En la siguiente función se definen las simulaciones de Monte Carlo de la misma manera que para Planck y posteriormente se adapta para poder ejecutarla en todos los modelos pudiendo seleccionar la frecuencia

La función MCDQui realiza inferencias utilizando el método MC Dropout sobre un conjunto de imágenes. MC Dropout se usa para estimar la incertidumbre en las predicciones del modlo, realizando múltiples pasadas de Dropout durante la inferencia.

In [30]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Número de pasadas de MC Dropout por imagen
# Para almacenar los resultados
def MCDQui(n_iter, path, model, data):
    resultados = []  # Lista para almacenar los resultados

    # Iterar sobre cada nombre de imagen en el DataFrame
    for name in data.name:
        try:
            # Construir la ruta completa del archivo de imagen
            filename = f'{name}.jpg'
            ruta_imagen = os.path.join(path, filename)
            
            # Cargar y preprocesar la imagen
            imagen = load_img(ruta_imagen, target_size=(150, 150))  # Ajusta el tamaño según tu modelo
            imagen = img_to_array(imagen) / 255.0  # Reescala si tu modelo lo requiere
            imagen = np.expand_dims(imagen, axis=0)  # Ajusta la forma para el modelo

            # Realizar n_iter pasadas de MC Dropout
            predicciones = np.array([model(imagen, training=True)[0] for _ in range(n_iter)])

            # Calcular estadísticas
            media = np.mean(predicciones, axis=0)
            std = np.std(predicciones, axis=0)

            # Guardar resultados
            resultados.append({
                'name': name,
                'media': media,
                'std': std,
            })
        except FileNotFoundError:
            continue  # Si la imagen no se encuentra, se omite y se continúa con la siguiente

    # Convertir la lista de resultados en un DataFrame
    df_resultados = pd.DataFrame(resultados)
    
    return df_resultados


In [31]:
def test_quijote(frec):
    names = []
    accs = []
    for file_name in os.listdir('../models/'):
            # Filtrar por archivos .h5 que no contienen 'VI' en su nombre
        if file_name.endswith('.h5') and 'VI' not in file_name:
            # Cargar el modelo de Keras
            model_path = os.path.join('../models/', file_name)
            model = tf.keras.models.load_model(model_path)
            #Ejecución de la simulación
            qui = MCDQui(2,f'../images/quijote/images_quijote_{frec}/',model,main)
            # Crear el DataFrame del resumen del modelo
            qui['OK'] = np.where(qui.media > 0.5,1,0)
            name  = file_name.replace('.h5', '')
            acc = len(qui.loc[qui.OK == 1])/len(qui)
            names.append(name)
            accs.append(acc)
            df = pd.DataFrame({'model':names,'Accuracy': accs})
    return df
        


## Ejecución de simulaciones en todos los modelos

Para evaluar el rendimiento de todos los modelos sobre QUIJOTE se ejecuta la función anterior en las 4 frecuencias

In [ ]:
test11 = test_quijote(11)
test13 = test_quijote(13)
test17 = test_quijote(17)
test19 = test_quijote(19)

In [14]:
test11 = test11.rename(columns = {'Accuracy':'Accuracy 11 GHz'})
test13 = test13.rename(columns = {'Accuracy':'Accuracy 13 GHz'})
test17 = test17.rename(columns = {'Accuracy':'Accuracy 17 GHz'})
test19 = test19.rename(columns = {'Accuracy':'Accuracy 19 GHz'})

In [15]:
test = pd.merge(test11,test13,on = 'model')
test = pd.merge(test,test17,on = 'model')
test = pd.merge(test,test19,on = 'model')

In [23]:
test['Accuracy'] = test.apply(lambda row: np.mean([row['Accuracy 11 GHz'], 
                                                   row['Accuracy 13 GHz'], 
                                                   row['Accuracy 17 GHz'], 
                                                   row['Accuracy 19 GHz']]), axis=1)

In [27]:
test = test.sort_values(by = 'Accuracy', ascending = False)

In [31]:
test.to_csv('../experiments/results/quijote/test_qui.csv',sep = ';')

## Simulaciones con el modelo final

Se realiza una última prueba con más iteraciones con el modelo final para obtener medidas más fiables sobre la media y la incertidumbre

In [ ]:
results11_ext = MCDQui(200,'..images/quijote/images_quijote_11_ext/',model,main)
results11 = MCDQui(200,'..images/quijote/images_quijote_11/',model,main)

results13_ext = MCDQui(200,'..images/quijote/images_quijote_13_ext/',model,ext)
results13 = MCDQui(200,'..images/quijote/images_quijote_13/',model,main)

results17_ext = MCDQui(200,'..images/quijote/images_quijote_17_ext/',model,ext)
results17 = MCDQui(200,'..images/quijote/images_quijote_17/',model,main)

results19_ext = MCDQui(200,'..images/quijote/images_quijote_19_ext/',model,ext)
results19 = MCDQui(200,'..images/quijote/images_quijote_19/',model,main)